# SMIB system as in Milano's book example 8.1

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sopt
import ipywidgets
from pydae import ssa
import control as ctrl

## Import system module

In [3]:
from smib_milano_ex8p1_4ord import smib_milano_ex8p1_4ord_class

## Instantiate system

In [4]:
syst = smib_milano_ex8p1_4ord_class()
syst.t_end = 15.0
syst.Dt = 0.001
syst.decimation =1
syst.update()

## Initialize the system (backward and foreward)

In [5]:
events=[{}]
syst.initialize(events,xy0=1)

(array([], shape=(0, 1), dtype=float64),
 array([], shape=(0, 4), dtype=float64),
 array([], shape=(0, 6), dtype=float64),
 array([], shape=(0, 2), dtype=float64))

In [6]:
syst.struct.Hy

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.72663535,  0.70707554,  0.79276616, -0.20000015,
          0.        ,  0.        ]]])

## Run in two time intervals

In [64]:
events=[{'t_end':1.0}]
syst.run(events)
events=[{'t_end':2.0}]
syst.run(events)

1

In [65]:
syst.inputs_run_list

['p_m', 'v_f']

In [66]:
self = syst




In [67]:
syst.get_value('omega')

4.5127034567025355

In [68]:
syst.set_value('p_m', 0.0)

In [69]:
syst.get_value('p_m')

68.55079227969233

In [112]:
Ts_control = 0.010
times = np.arange(0.0,10,Ts_control)

# Calculate second references
events=[{'P_t':0.9, 'Q_t':0.0}]
syst.initialize(events,xy0=1.0)
x_ref = np.copy(syst.struct[0].x)
v_f_ref = syst.struct[0]['v_f']
p_m_ref = syst.struct[0]['p_m']

# Calculate initial references
events=[{'P_t':0.0, 'Q_t':0.0}]
syst.initialize(events,xy0=1.0)
x_0 = np.copy(syst.struct[0].x)
v_f_0 = syst.get_value('v_f')
p_m_0 = syst.get_value('p_m')

# Control design
ssa.eval_ss(syst)
Q = np.eye(syst.N_x)*100
R = np.eye(syst.N_u)

K = ctrl.place(syst.A,syst.B,[-2.0+1j*6,-2.0-1j*6,-100,-100])
K,S,E = ctrl.lqr(syst.A,syst.B,Q,R)
Ad,Bd = ssa.discretise_time(syst.A,syst.B,Ts_control)
Kd,S,E = ssa.dlqr(Ad,Bd,Q,R)

for t in times:
    
    x = np.copy(syst.struct[0].x)
    v_f = v_f_0 
    p_m = p_m_0 

    if t>1.0: 
        u_ctrl = K*(x_ref - x)
        p_m = p_m_ref + u_ctrl[0]
        v_f = v_f_ref + u_ctrl[1]
        
        
    events=[{'t_end':t,'v_f':v_f,'p_m':p_m}]
    syst.run(events)

syst.post();
    

In [113]:
plt.close('all')
fig, axes = plt.subplots(nrows=2,ncols=2, figsize=(10, 5), frameon=False, dpi=50)

axes[0,0].plot(syst.T, syst.get_values('omega'), label=f'$\omega$')
axes[0,1].plot(syst.T, syst.get_values('v_1'),   label=f'$v_1$')
axes[1,0].plot(syst.T, syst.get_values('P_t'),   label=f'$P_t$')
axes[1,1].plot(syst.T, syst.get_values('Q_t'),   label=f'$Q_t$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
ssa.eval_ss(syst)


array([[ 0.00000000e+00,  3.14159265e+02,  0.00000000e+00,
         0.00000000e+00],
       [-3.54343376e-01, -1.42857143e-01, -3.57095245e-01,
         1.00330478e-01],
       [-4.67038328e-01,  0.00000000e+00, -6.64265905e-01,
        -2.31113959e-03],
       [ 7.98653399e-01,  0.00000000e+00,  1.35913375e-02,
        -2.58565604e+00]])

In [33]:
from scipy.signal import ss2tf,lti,bode

In [37]:
num,den =ss2tf(syst.A,syst.B,syst.C,syst.D,input=0)

In [38]:
G = lti(num[1],den)

C:\programdata\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1622: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)


In [39]:
w, mag, phase = G.bode()
plt.figure()
plt.semilogx(w, mag)    # Bode magnitude plot
plt.figure()
plt.semilogx(w, phase)  # Bode phase plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
events=[{'t_end':1.0,'P_t':0.8, 'Q_t':0.5},
        {'t_end':10.0, 'p_m':0.9}
       ]

syst.simulate(events,xy0=1.0);

In [20]:
syst.inputs_run_list


['p_m', 'v_f']

In [43]:
0.01/6

0.0016666666666666668

In [28]:
syst.B

array([[0.        , 0.        ],
       [0.14285714, 0.        ],
       [0.        , 0.125     ],
       [0.        , 0.        ]])

In [30]:
syst.struct[0]['Fu']

array([[0.        , 0.        ],
       [0.14285714, 0.        ],
       [0.        , 0.125     ],
       [0.        , 0.        ]])